In [2]:
import networkx as nx
import random
import math
import copy

In [3]:
def euclid_distance(u_x: float,
                    u_y: float,
                    v_x: float,
                    v_y: float) -> float:
    return math.sqrt((u_x - v_x)**2 + (u_y - v_y)**2)

In [4]:
def read_instance(file_path: str) -> nx.Graph:
    with open(file_path, 'r') as f:
        g = nx.Graph()
        for line in f:
            if line[0].isdigit():
                node_id, x, y = [float(a) for a in line.split()]
                g.add_node(int(node_id) - 1, x=x, y=y)
        
        for u in range(len(g.nodes)):
            for v in range(u + 1, len(g.nodes)):
                
                weight = euclid_distance(g.nodes[u]['x'],
                                        g.nodes[u]['y'],
                                        g.nodes[v]['x'],
                                        g.nodes[v]['y'])
                
                g.add_edge(u, v, weight=weight)
                
        return g

In [5]:
g = read_instance(file_path='dj38.tsp')

In [266]:
class Harmony: 
    def __init__(self, start):
        self.cycle, self.fitness = self.hamiltonian(g, start)
    
    def __str__(self):
        return ' '.join([str(u) for u in self.cycle])# + f'{self.fitness}'
    
    def __getitem__(self,index):
        return self.cycle[index]
    
    def hamiltonian(
        self: Harmony,
        g: nx.Graph,
        start: int
    ):
    
        cycle = [start]
        cycle_weight = 0
        visited = {start}
        u = start
        while len(visited) != len(g.nodes):
            neighbours = [v for v in g[u] if v not in visited]
            chosen_neighbour = random.choices(neighbours, weights=None, k=1)[0]
            cycle.append(chosen_neighbour)
            visited.add(chosen_neighbour)
            cycle_weight += g[u][chosen_neighbour]['weight']
            u = chosen_neighbour
        
        cycle_weight += g[start][cycle[-1]]['weight']
        return cycle, cycle_weight

In [283]:
def HarmonySearch( #TODO 1)same random?
    g: nx.Graph,
    hms: int, #number of harmonies
    rA: float, #harmony memory accepting rate
    rPA: float, #pitch adjusting rate
    maxIterations: int
):
    HarmonyMemory = [Harmony(start=0) for _ in range(hms)]
    
    for t in range(maxIterations):
        numOfCities = len(HarmonyMemory[0])
        
        newHarmony = []
        for i in range(numOfCities):
            if random.random() < rA:
                #chose a value from HM for the variable i (play same harmony)
                newHarmony = chooseFromHM(HarmonyMemory, newHarmony, i)
                if random.random() < rPA:
                    #adjust the value (improvise on harmony)
                    newHarmony = adjustHarmony(newHarmony)
            else:
                #chose random harmony (improvise)
                newHarmony.append(random.choices(range(), 1))
            
            if len(newHarmony) != i+1 : #node doesnt have unvisited neighbor
                break
            
        #accept solution if better than worst
        HarmonyMemory.sort(key=lambda x : x[_].fitness)
        
        #(calc fitness and sort hm fitnesses)
        
    #return best solution

In [286]:
for h in HarmonyMemory:
    print(h.fitness)

26974.899782271918
28507.750690001056
27995.74459608851
26373.08636891863
26857.525465089122


In [1]:
HarmonyMemory.sort(key= lambda x : (HarmonyMemory[x].fitness)) #todo

NameError: name 'HarmonyMemory' is not defined

In [273]:
def chooseFromHM(HarmonyMemory : list, harmony : list, i : int) -> list:
    candidates = [HarmonyMemory[u][i] for u in range(len(HarmonyMemory))]
    
    random.shuffle(candidates)
    
    for c in candidates:
        if c not in harmony:
            if len(harmony) == 0 or g.has_edge(harmony[-1], c):
                harmony.append(c)
                return harmony
    return harmony

In [158]:
def adjustHarmony(harmony : list) -> list:
    var = harmony[-1]
    neighbors = [u for u in g.neighbors(var)]
    
    random.shuffle(neighbors)
    
    for u in neighbors:
        if u not in harmony:
            if len(harmony) == 1 or g.has_edge(harmony[-2], u):
                harmony.remove(var)
                harmony.append(u)
                return harmony
    return harmony ##problem?

In [281]:
def improvise(harmony : list) -> list:
    neighbors = [u for u in g.neighbors(var)]
    
    random.shuffle(neighbors)
    
    for u in neighbors:
        if u not in harmony:
            if len(harmony) == 0 or g.has_edge(harmony[-1], u):
                harmony.append(u)
                return harmony
    return harmony #problem?